Этот блокнот - про важность оси времени. 

Класс ```TimeWeightWrapper``` применяет к данным веса, убывающие со временем, и обучает с этими весами ваш любимый агоритм.

Если в данных есть тренды, взвешивание наблюдений по релевантности - один из самых простых способов учесть эти тренды. 

In [1]:
from sklearn.base import BaseEstimator

class TimeWeightWrapper(BaseEstimator):
    """ Apply exponential decay to observation weights 
        before fitting the base model.
        Assume the first observations are the oldest.
    """
    def __init__(self, base, decay=0.99):
        self.base = base
        self.decay = decay
    def fit(self, X, y, initial_weights=1):
        sample_weights = initial_weights * self.decay ** -np.arange(len(y))
        self.base.fit(X, y, sample_weight=sample_weights)
    def predict(self, X):
        return self.base.predict(X)
    def score(self, X, y):
        return self.base.score(X, y)
# a bug: if you use cross-validation with n_jobs > 1, this class should be put into a .py file.

Создадим обычную iid выборку, и добавим в неё немножко линейного тренда. 

Первое правило: если данные упорядочены по времени, используйте кросс-валидацию ```TimeSeriesSplit``` вместо дефолтной ```KFold```. Ибо ```KFold``` непременно завысит точность ваших прогнозов будущего. 

Второе правило: если не хотите включать время в модель в явном виде, перевзвешивайте наблюдения. Это здорово поднимет точность. Особенно если затюнить коэффициент затухания. 

Третье правило: если время важно, всё-таки включите его в модель. В данном примере в $y$ был включён линейный тренд, и модель его благополучно поймала. Далеко не всегда вас это спасёт, потому что вовсе не факт, что эффект времени линейный. Но попробовать стоит.

In [2]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.linear_model import Ridge

# create toy data
X, y = make_regression(n_samples=1000, noise=100, random_state=1)
# add a linear trend to the data
y += np.linspace(0, 300, num=y.shape[0])

# ordinary cross-validation shows R2 biased upward
print('Ordinary CV \t{:3.1%}'.format(cross_val_score(Ridge(1), X, y, cv=5).mean()))
# time-ordered decay shows more realistic R2
tss = TimeSeriesSplit(5)
print('Time-ordered \t{:3.1%}'.format(cross_val_score(Ridge(1), X, y, cv=tss).mean()))
# reweighing observations increases performance
print('With decay \t{:3.1%}'.format(cross_val_score(TimeWeightWrapper(Ridge(1)), X, y, cv=tss).mean()))
# tuning decay rate boosts R2 even more
print('Tuned decay \t{:3.1%}'.format(cross_val_score(TimeWeightWrapper(Ridge(1), 0.9935), X, y, cv=tss).mean()))
# build a model that explicitly uses time trend
print('Explicit trend \t{:3.1%}'.format(cross_val_score(Ridge(1), np.hstack([X, np.arange(1000)[:, np.newaxis]]), y, cv=5).mean()))

Ordinary CV 	56.1%
Time-ordered 	49.7%
With decay 	56.9%
Tuned decay 	58.1%
Explicit trend 	78.6%
